### Loading Data

In [145]:
from sklearn.model_selection import train_test_split
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

In [94]:
path = r"C:\Users\wayiqin\Downloads\machine_learning\Assignment5\crx.csv" #change path for linux
dataset = pd.read_csv(path,header=None,sep='\t', delimiter=',',names=["data"])
dataset.head(10)
splitdata = dataset["data"].str.split(",")
data = splitdata.to_list()
names = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11", "A12", "A13", "A14", "A15", "A16"]
new_df = pd.DataFrame(data, columns=names)
new_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0,u,g,w,v,1.25,t,t,01,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,06,f,g,00043,560,+
2,a,24.50,0.5,u,g,q,h,1.5,t,f,0,f,g,00280,824,+
3,b,27.83,1.54,u,g,w,v,3.75,t,t,05,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.75,u,g,c,v,2,f,t,02,t,g,00200,394,-
687,a,25.25,13.5,y,p,ff,ff,2,f,t,01,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


### Checking for null values

In [77]:
print(new_df.isnull().any())

A1     False
A2     False
A3     False
A4     False
A5     False
A6     False
A7     False
A8     False
A9     False
A10    False
A11    False
A12    False
A13    False
A14    False
A15    False
A16    False
dtype: bool


### Checking for meaningless values

In [78]:
new_df.A1.value_counts()

b    468
a    210
?     12
Name: A1, dtype: int64

In [95]:
new_df = new_df[new_df[new_df.columns] != '?']

In [80]:
new_df.A1.value_counts()

b    468
a    210
Name: A1, dtype: int64

In [96]:
new_df_nonan = new_df.dropna(how = 'any') 
new_df_nonan

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0,u,g,w,v,1.25,t,t,01,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,06,f,g,00043,560,+
2,a,24.50,0.5,u,g,q,h,1.5,t,f,0,f,g,00280,824,+
3,b,27.83,1.54,u,g,w,v,3.75,t,t,05,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.75,u,g,c,v,2,f,t,02,t,g,00200,394,-
687,a,25.25,13.5,y,p,ff,ff,2,f,t,01,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [97]:
new_df_nonan.reset_index(inplace=True,drop=True)
new_df_nonan

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0,u,g,w,v,1.25,t,t,01,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,06,f,g,00043,560,+
2,a,24.50,0.5,u,g,q,h,1.5,t,f,0,f,g,00280,824,+
3,b,27.83,1.54,u,g,w,v,3.75,t,t,05,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
649,a,22.67,0.75,u,g,c,v,2,f,t,02,t,g,00200,394,-
650,a,25.25,13.5,y,p,ff,ff,2,f,t,01,t,g,00200,1,-
651,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


### Convert categorical variables into numeric variables

In [99]:
def A1_to_numeric(x):
    if x=='a':
        return 1
    if x=='b':
        return 2
def A4_to_numeric(x):
    if x=='u':
        return 1
    if x=='y':
        return 2
    if x=='l':
        return 3
    if x=='t':
        return 4
def A5_to_numeric(x):
    if x=='g':
        return 1
    if x=='p':
        return 2
    if x=='gg':
        return 3
def A6_to_numeric(x):
    if x=='c':
        return 1
    if x=='d':
        return 2
    if x=='cc':
        return 3
    if x=='i':
        return 4
    if x=='j':
        return 5
    if x=='k':
        return 6
    if x=='m':
        return 7
    if x=='r':
        return 8
    if x=='q':
        return 9
    if x=='w':
        return 10
    if x=='x':
        return 11
    if x=='e':
        return 12
    if x=='aa':
        return 13
    if x=='ff':
        return 14
def A7_to_numeric(x):
    if x=='v':
        return 1
    if x=='h':
        return 2
    if x=='bb':
        return 3
    if x=='j':
        return 4
    if x=='n':
        return 5
    if x=='z':
        return 6
    if x=='dd':
        return 7
    if x=='ff':
        return 8
    if x=='o':
        return 9
def A9_10_12_to_numeric(x):
    if x=='t':
        return 1
    if x=='f':
        return 2
def A13_to_numeric(x):
    if x=='g':
        return 1
    if x=='p':
        return 2
    if x=='s':
        return 3

In [100]:
new_df_nonan.loc[:,'A1']=new_df_nonan.loc[:,'A1'].apply(A1_to_numeric)
new_df_nonan.loc[:,'A4']=new_df_nonan.loc[:,'A4'].apply(A4_to_numeric)
new_df_nonan.loc[:,'A5']=new_df_nonan.loc[:,'A5'].apply(A5_to_numeric)
new_df_nonan.loc[:,'A6']=new_df_nonan.loc[:,'A6'].apply(A6_to_numeric)
new_df_nonan.loc[:,'A7']=new_df_nonan.loc[:,'A7'].apply(A7_to_numeric)
new_df_nonan.loc[:,'A9']=new_df_nonan.loc[:,'A9'].apply(A9_10_12_to_numeric)
new_df_nonan.loc[:,'A10']=new_df_nonan.loc[:,'A10'].apply(A9_10_12_to_numeric)
new_df_nonan.loc[:,'A12']=new_df_nonan.loc[:,'A12'].apply(A9_10_12_to_numeric)
new_df_nonan.loc[:,'A13']=new_df_nonan.loc[:,'A13'].apply(A13_to_numeric)
new_df_nonan

C:\Users\wayiqin\Documents\Anaconda\envs\pythonenv\lib\site-packages\pandas\core\indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\wayiqin\Documents\Anaconda\envs\pythonenv\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,30.83,0,1,1,10,1,1.25,1,1,01,2,1,00202,0,+
1,1,58.67,4.46,1,1,9,2,3.04,1,1,06,2,1,00043,560,+
2,1,24.50,0.5,1,1,9,2,1.5,1,2,0,2,1,00280,824,+
3,2,27.83,1.54,1,1,10,1,3.75,1,1,05,1,1,00100,3,+
4,2,20.17,5.625,1,1,10,1,1.71,1,2,0,2,3,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,2,21.08,10.085,2,2,12,2,1.25,2,2,0,2,1,00260,0,-
649,1,22.67,0.75,1,1,1,1,2,2,1,02,1,1,00200,394,-
650,1,25.25,13.5,2,2,14,8,2,2,1,01,1,1,00200,1,-
651,2,17.92,0.205,1,1,13,1,0.04,2,2,0,2,1,00280,750,-


### Define ANN Model

In [150]:
#Define the ANN
ANNModel = Sequential() #Sequential ANN

#Layers of the ANN (Dense in this case)
ANNModel.add(Dense(30, input_dim=15, activation='relu'))
ANNModel.add(Dense(3, input_dim=30, activation='relu'))
ANNModel.add(Dense(1, input_dim=3, activation='sigmoid'))
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
ANNModel.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
ANNModel.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 30)                480       
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 93        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 4         
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________


In [116]:
## split dataset into features and target
variables = new_df_nonan.iloc[:,0:15]
target = new_df_nonan.iloc[:,15]

scalar = MinMaxScaler()
scalar.fit(variables)
variables = scalar.transform(variables)

def target_to_numeric(x):
    if x=='+':
        return 0
    if x=='-':
        return 1

## convert target into numeric
target = target.apply(target_to_numeric)
print(target)

0      0
1      0
2      0
3      0
4      0
      ..
648    1
649    1
650    1
651    1
652    1
Name: A16, Length: 653, dtype: int64


In [117]:
X_train, X_test, y_train, y_test = train_test_split(variables, target, test_size=0.33, random_state=42)

In [151]:
nb_epoch = 200 
batch_size = 32

#Get the ANN parameters                   
ANNModel.fit(X_train, y_train,
              batch_size = batch_size,
              epochs = nb_epoch)

Epoch 1/200
437/437 [==============================] - 0s 208us/step - loss: 0.6125 - accuracy: 0.6133
Epoch 2/200
437/437 [==============================] - 0s 32us/step - loss: 0.4794 - accuracy: 0.7963
Epoch 3/200
437/437 [==============================] - 0s 30us/step - loss: 0.3766 - accuracy: 0.8764
Epoch 4/200
437/437 [==============================] - 0s 30us/step - loss: 0.3357 - accuracy: 0.8787
Epoch 5/200
437/437 [==============================] - 0s 34us/step - loss: 0.3407 - accuracy: 0.8719
Epoch 6/200
437/437 [==============================] - 0s 25us/step - loss: 0.3213 - accuracy: 0.8787
Epoch 7/200
437/437 [==============================] - 0s 27us/step - loss: 0.3049 - accuracy: 0.8902
Epoch 8/200
437/437 [==============================] - 0s 32us/step - loss: 0.3001 - accuracy: 0.8810
Epoch 9/200
437/437 [==============================] - 0s 32us/step - loss: 0.2959 - accuracy: 0.8764
Epoch 10/200
437/437 [==============================] - 0s 32us/step - loss: 0.29

437/437 [==============================] - 0s 30us/step - loss: 0.0501 - accuracy: 0.9863
Epoch 159/200
437/437 [==============================] - 0s 27us/step - loss: 0.0494 - accuracy: 0.9908
Epoch 160/200
437/437 [==============================] - 0s 30us/step - loss: 0.0478 - accuracy: 0.9908
Epoch 161/200
437/437 [==============================] - 0s 30us/step - loss: 0.0465 - accuracy: 0.9863
Epoch 162/200
437/437 [==============================] - 0s 27us/step - loss: 0.0486 - accuracy: 0.9863
Epoch 163/200
437/437 [==============================] - 0s 25us/step - loss: 0.0501 - accuracy: 0.9931
Epoch 164/200
437/437 [==============================] - 0s 27us/step - loss: 0.0473 - accuracy: 0.9863
Epoch 165/200
437/437 [==============================] - 0s 27us/step - loss: 0.0614 - accuracy: 0.9794
Epoch 166/200
437/437 [==============================] - 0s 30us/step - loss: 0.0619 - accuracy: 0.9794
Epoch 167/200
437/437 [==============================] - 0s 30us/step - loss: 

In [152]:
y_pred = ANNModel.predict_classes(X_test)
y_test.reset_index(inplace=True,drop=True)
# print(y_pred)
# print(y_test)
# show the inputs and predicted outputs
predy = []
testy = []
for i in range(len(y_pred)):
    predy.append(y_pred[i][0])
    testy.append(y_test.loc[i])
    print("Truth=", y_test.loc[i], "Predicted=", y_pred[i][0])

Truth= 1 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 1 Predicted= 0
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 1 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 1
Truth= 0 Predicted= 0
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 0 Predicted= 1
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 0 Predicted= 1
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 1 Predicted= 1
Truth= 0 Predicted= 0
Truth= 0 P

### Evaluate the result

In [153]:
accuracy = accuracy_score(testy, predy)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, predy)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, predy)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, predy)
print('F1 score: %f' % f1)

Accuracy: 0.842593
Precision: 0.865546
Recall: 0.851240
F1 score: 0.858333


### Increase the Number of epochs

In [154]:
nb_epoch = 500 
batch_size = 32

#Get the ANN parameters                   
ANNModel.fit(X_train, y_train,
              batch_size = batch_size,
              epochs = nb_epoch)

y_pred = ANNModel.predict_classes(X_test)

# print(y_pred)
# print(y_test)

predy = []
testy = []
for i in range(len(y_pred)):
    predy.append(y_pred[i][0])
    testy.append(y_test.loc[i])

Epoch 1/500
437/437 [==============================] - 0s 30us/step - loss: 0.0652 - accuracy: 0.9748
Epoch 2/500
437/437 [==============================] - 0s 48us/step - loss: 0.0934 - accuracy: 0.9703
Epoch 3/500
437/437 [==============================] - 0s 39us/step - loss: 0.0495 - accuracy: 0.9794
Epoch 4/500
437/437 [==============================] - 0s 39us/step - loss: 0.0443 - accuracy: 0.9863
Epoch 5/500
437/437 [==============================] - 0s 39us/step - loss: 0.0370 - accuracy: 0.9840
Epoch 6/500
437/437 [==============================] - 0s 34us/step - loss: 0.0353 - accuracy: 0.9908
Epoch 7/500
437/437 [==============================] - 0s 39us/step - loss: 0.0474 - accuracy: 0.9771
Epoch 8/500
437/437 [==============================] - 0s 34us/step - loss: 0.0415 - accuracy: 0.9840
Epoch 9/500
437/437 [==============================] - 0s 37us/step - loss: 0.0492 - accuracy: 0.9840
Epoch 10/500
437/437 [==============================] - 0s 34us/step - loss: 0.038

437/437 [==============================] - 0s 32us/step - loss: 0.0115 - accuracy: 0.9954
Epoch 160/500
437/437 [==============================] - 0s 30us/step - loss: 0.0110 - accuracy: 0.9954
Epoch 161/500
437/437 [==============================] - 0s 27us/step - loss: 0.0112 - accuracy: 0.9954
Epoch 162/500
437/437 [==============================] - 0s 32us/step - loss: 0.0099 - accuracy: 0.9954
Epoch 163/500
437/437 [==============================] - 0s 25us/step - loss: 0.0116 - accuracy: 0.9931
Epoch 164/500
437/437 [==============================] - 0s 27us/step - loss: 0.0134 - accuracy: 0.9977
Epoch 165/500
437/437 [==============================] - 0s 27us/step - loss: 0.0113 - accuracy: 0.9954
Epoch 166/500
437/437 [==============================] - 0s 30us/step - loss: 0.0114 - accuracy: 0.9977
Epoch 167/500
437/437 [==============================] - 0s 34us/step - loss: 0.0115 - accuracy: 0.9931
Epoch 168/500
437/437 [==============================] - 0s 27us/step - loss: 

437/437 [==============================] - 0s 30us/step - loss: 0.0072 - accuracy: 0.9977
Epoch 316/500
437/437 [==============================] - 0s 32us/step - loss: 0.0071 - accuracy: 0.9977
Epoch 317/500
437/437 [==============================] - 0s 32us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 318/500
437/437 [==============================] - 0s 27us/step - loss: 0.0076 - accuracy: 0.9977
Epoch 319/500
437/437 [==============================] - ETA: 0s - loss: 0.0393 - accuracy: 0.96 - 0s 30us/step - loss: 0.0124 - accuracy: 0.9908
Epoch 320/500
437/437 [==============================] - 0s 27us/step - loss: 0.0094 - accuracy: 0.9977
Epoch 321/500
437/437 [==============================] - 0s 32us/step - loss: 0.0073 - accuracy: 0.9977
Epoch 322/500
437/437 [==============================] - 0s 32us/step - loss: 0.0082 - accuracy: 0.9954
Epoch 323/500
437/437 [==============================] - 0s 30us/step - loss: 0.0062 - accuracy: 0.9977
Epoch 324/500
437/437 [=============

437/437 [==============================] - 0s 27us/step - loss: 0.0056 - accuracy: 0.9977
Epoch 394/500
437/437 [==============================] - 0s 27us/step - loss: 0.0075 - accuracy: 0.9977
Epoch 395/500
437/437 [==============================] - 0s 27us/step - loss: 0.0070 - accuracy: 0.9977
Epoch 396/500
437/437 [==============================] - 0s 27us/step - loss: 0.0061 - accuracy: 0.9977
Epoch 397/500
437/437 [==============================] - 0s 30us/step - loss: 0.0071 - accuracy: 0.9954
Epoch 398/500
437/437 [==============================] - 0s 30us/step - loss: 0.0118 - accuracy: 0.9977
Epoch 399/500
437/437 [==============================] - 0s 27us/step - loss: 0.0157 - accuracy: 0.9954
Epoch 400/500
437/437 [==============================] - 0s 30us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 401/500
437/437 [==============================] - 0s 27us/step - loss: 0.0070 - accuracy: 0.9977
Epoch 402/500
437/437 [==============================] - 0s 30us/step - loss: 

In [155]:
accuracy = accuracy_score(testy, predy)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, predy)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, predy)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, predy)
print('F1 score: %f' % f1)

Accuracy: 0.856481
Precision: 0.875000
Recall: 0.867769
F1 score: 0.871369


### Increase the Learning rate

In [156]:
#Define the ANN
ANNModel = Sequential() #Sequential ANN

#Layers of the ANN (Dense in this case)
ANNModel.add(Dense(30, input_dim=15, activation='relu'))
ANNModel.add(Dense(3, input_dim=30, activation='relu'))
ANNModel.add(Dense(1, input_dim=3, activation='sigmoid'))
adam = optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
ANNModel.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
ANNModel.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 30)                480       
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 93        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 4         
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________


In [157]:
nb_epoch = 200 
batch_size = 32

#Get the ANN parameters                   
ANNModel.fit(X_train, y_train,
              batch_size = batch_size,
              epochs = nb_epoch)

y_pred = ANNModel.predict_classes(X_test)

# print(y_pred)
# print(y_test)

predy = []
testy = []
for i in range(len(y_pred)):
    predy.append(y_pred[i][0])
    testy.append(y_test.loc[i])

Epoch 1/200
437/437 [==============================] - 0s 199us/step - loss: 0.5888 - accuracy: 0.6064
Epoch 2/200
437/437 [==============================] - 0s 32us/step - loss: 0.4013 - accuracy: 0.8741
Epoch 3/200
437/437 [==============================] - 0s 30us/step - loss: 0.3708 - accuracy: 0.8764
Epoch 4/200
437/437 [==============================] - 0s 34us/step - loss: 0.3472 - accuracy: 0.8719
Epoch 5/200
437/437 [==============================] - 0s 27us/step - loss: 0.3371 - accuracy: 0.8764
Epoch 6/200
437/437 [==============================] - 0s 32us/step - loss: 0.3798 - accuracy: 0.8535
Epoch 7/200
437/437 [==============================] - 0s 32us/step - loss: 0.3244 - accuracy: 0.8879
Epoch 8/200
437/437 [==============================] - 0s 34us/step - loss: 0.3277 - accuracy: 0.8856
Epoch 9/200
437/437 [==============================] - 0s 32us/step - loss: 0.3248 - accuracy: 0.8924
Epoch 10/200
437/437 [==============================] - 0s 27us/step - loss: 0.31

437/437 [==============================] - 0s 34us/step - loss: 0.2213 - accuracy: 0.9199
Epoch 159/200
437/437 [==============================] - 0s 30us/step - loss: 0.2280 - accuracy: 0.9245
Epoch 160/200
437/437 [==============================] - 0s 25us/step - loss: 0.2198 - accuracy: 0.9222
Epoch 161/200
437/437 [==============================] - 0s 27us/step - loss: 0.1971 - accuracy: 0.9268
Epoch 162/200
437/437 [==============================] - 0s 27us/step - loss: 0.2057 - accuracy: 0.9359
Epoch 163/200
437/437 [==============================] - 0s 30us/step - loss: 0.1946 - accuracy: 0.9291
Epoch 164/200
437/437 [==============================] - 0s 27us/step - loss: 0.2087 - accuracy: 0.9291
Epoch 165/200
437/437 [==============================] - 0s 25us/step - loss: 0.2042 - accuracy: 0.9222
Epoch 166/200
437/437 [==============================] - 0s 27us/step - loss: 0.2046 - accuracy: 0.9245
Epoch 167/200
437/437 [==============================] - 0s 25us/step - loss: 

In [158]:
accuracy = accuracy_score(testy, predy)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, predy)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, predy)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, predy)
print('F1 score: %f' % f1)

Accuracy: 0.856481
Precision: 0.881356
Recall: 0.859504
F1 score: 0.870293


### Change the numer of layer

In [159]:
#Define the ANN
ANNModel = Sequential() #Sequential ANN

#Layers of the ANN (Dense in this case)
ANNModel.add(Dense(30, input_dim=15, activation='relu'))
ANNModel.add(Dense(20, input_dim=30, activation='relu'))
ANNModel.add(Dense(10, input_dim=20, activation='relu'))
ANNModel.add(Dense(3, input_dim=10, activation='relu'))
ANNModel.add(Dense(1, input_dim=3, activation='sigmoid'))
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
ANNModel.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
ANNModel.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 30)                480       
_________________________________________________________________
dense_32 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_33 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_34 (Dense)             (None, 3)                 33        
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 4         
Total params: 1,347
Trainable params: 1,347
Non-trainable params: 0
_________________________________________________________________


In [160]:
nb_epoch = 200 
batch_size = 32

#Get the ANN parameters                   
ANNModel.fit(X_train, y_train,
              batch_size = batch_size,
              epochs = nb_epoch)

y_pred = ANNModel.predict_classes(X_test)

# print(y_pred)
# print(y_test)

predy = []
testy = []
for i in range(len(y_pred)):
    predy.append(y_pred[i][0])
    testy.append(y_test.loc[i])

Epoch 1/200
437/437 [==============================] - 0s 274us/step - loss: 0.5302 - accuracy: 0.6636
Epoch 2/200
437/437 [==============================] - 0s 39us/step - loss: 0.4509 - accuracy: 0.8787
Epoch 3/200
437/437 [==============================] - 0s 41us/step - loss: 0.4337 - accuracy: 0.8741
Epoch 4/200
437/437 [==============================] - 0s 39us/step - loss: 0.4187 - accuracy: 0.8741
Epoch 5/200
437/437 [==============================] - 0s 36us/step - loss: 0.3913 - accuracy: 0.8764
Epoch 6/200
437/437 [==============================] - 0s 34us/step - loss: 0.3794 - accuracy: 0.8787
Epoch 7/200
437/437 [==============================] - 0s 37us/step - loss: 0.3633 - accuracy: 0.8787
Epoch 8/200
437/437 [==============================] - 0s 37us/step - loss: 0.3643 - accuracy: 0.8856
Epoch 9/200
437/437 [==============================] - 0s 37us/step - loss: 0.3530 - accuracy: 0.8764
Epoch 10/200
437/437 [==============================] - 0s 36us/step - loss: 0.35

437/437 [==============================] - 0s 34us/step - loss: 0.1099 - accuracy: 0.9703
Epoch 160/200
437/437 [==============================] - 0s 30us/step - loss: 0.1076 - accuracy: 0.9680
Epoch 161/200
437/437 [==============================] - 0s 34us/step - loss: 0.1067 - accuracy: 0.9703
Epoch 162/200
437/437 [==============================] - 0s 32us/step - loss: 0.1055 - accuracy: 0.9703
Epoch 163/200
437/437 [==============================] - 0s 32us/step - loss: 0.1051 - accuracy: 0.9725
Epoch 164/200
437/437 [==============================] - 0s 32us/step - loss: 0.1046 - accuracy: 0.9725
Epoch 165/200
437/437 [==============================] - 0s 30us/step - loss: 0.1041 - accuracy: 0.9725
Epoch 166/200
437/437 [==============================] - 0s 30us/step - loss: 0.1043 - accuracy: 0.9748
Epoch 167/200
437/437 [==============================] - 0s 30us/step - loss: 0.1045 - accuracy: 0.9725
Epoch 168/200
437/437 [==============================] - 0s 30us/step - loss: 

In [161]:
accuracy = accuracy_score(testy, predy)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, predy)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, predy)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, predy)
print('F1 score: %f' % f1)

Accuracy: 0.870370
Precision: 0.897436
Recall: 0.867769
F1 score: 0.882353


In [162]:
#Define the ANN
ANNModel = Sequential() #Sequential ANN

#Layers of the ANN (Dense in this case)
ANNModel.add(Dense(50, input_dim=15, activation='relu'))
ANNModel.add(Dense(10, input_dim=50, activation='relu'))
ANNModel.add(Dense(1, input_dim=10, activation='sigmoid'))
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
ANNModel.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
ANNModel.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 50)                800       
_________________________________________________________________
dense_37 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 11        
Total params: 1,321
Trainable params: 1,321
Non-trainable params: 0
_________________________________________________________________


In [163]:
nb_epoch = 200 
batch_size = 32

#Get the ANN parameters                   
ANNModel.fit(X_train, y_train,
              batch_size = batch_size,
              epochs = nb_epoch)

y_pred = ANNModel.predict_classes(X_test)

# print(y_pred)
# print(y_test)

predy = []
testy = []
for i in range(len(y_pred)):
    predy.append(y_pred[i][0])
    testy.append(y_test.loc[i])

Epoch 1/200
437/437 [==============================] - 0s 201us/step - loss: 0.5535 - accuracy: 0.7185
Epoch 2/200
437/437 [==============================] - 0s 30us/step - loss: 0.3560 - accuracy: 0.8696
Epoch 3/200
437/437 [==============================] - 0s 34us/step - loss: 0.3332 - accuracy: 0.8741
Epoch 4/200
437/437 [==============================] - 0s 34us/step - loss: 0.3200 - accuracy: 0.8764
Epoch 5/200
437/437 [==============================] - 0s 32us/step - loss: 0.3050 - accuracy: 0.8810
Epoch 6/200
437/437 [==============================] - 0s 30us/step - loss: 0.3076 - accuracy: 0.8856
Epoch 7/200
437/437 [==============================] - 0s 34us/step - loss: 0.2998 - accuracy: 0.8947
Epoch 8/200
437/437 [==============================] - 0s 34us/step - loss: 0.2903 - accuracy: 0.8902
Epoch 9/200
437/437 [==============================] - 0s 34us/step - loss: 0.2776 - accuracy: 0.8902
Epoch 10/200
437/437 [==============================] - 0s 34us/step - loss: 0.27

437/437 [==============================] - 0s 30us/step - loss: 0.3892 - accuracy: 0.8947
Epoch 160/200
437/437 [==============================] - 0s 34us/step - loss: 0.1550 - accuracy: 0.9359
Epoch 161/200
437/437 [==============================] - 0s 30us/step - loss: 0.0970 - accuracy: 0.9611
Epoch 162/200
437/437 [==============================] - 0s 25us/step - loss: 0.0697 - accuracy: 0.9748
Epoch 163/200
437/437 [==============================] - 0s 27us/step - loss: 0.0568 - accuracy: 0.9840
Epoch 164/200
437/437 [==============================] - 0s 30us/step - loss: 0.0442 - accuracy: 0.9886
Epoch 165/200
437/437 [==============================] - 0s 27us/step - loss: 0.0387 - accuracy: 0.9931
Epoch 166/200
437/437 [==============================] - 0s 27us/step - loss: 0.0324 - accuracy: 0.9908
Epoch 167/200
437/437 [==============================] - 0s 27us/step - loss: 0.0284 - accuracy: 0.9908
Epoch 168/200
437/437 [==============================] - 0s 25us/step - loss: 

In [164]:
accuracy = accuracy_score(testy, predy)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, predy)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, predy)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, predy)
print('F1 score: %f' % f1)

Accuracy: 0.856481
Precision: 0.875000
Recall: 0.867769
F1 score: 0.871369


In [165]:
#Define the ANN
ANNModel = Sequential() #Sequential ANN

#Layers of the ANN (Dense in this case)
ANNModel.add(Dense(30, input_dim=15, activation='sigmoid'))
ANNModel.add(Dense(3, input_dim=30, activation='sigmoid'))
ANNModel.add(Dense(1, input_dim=3, activation='sigmoid'))
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
ANNModel.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
ANNModel.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 30)                480       
_________________________________________________________________
dense_40 (Dense)             (None, 3)                 93        
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 4         
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________


In [166]:
nb_epoch = 200 
batch_size = 32

#Get the ANN parameters                   
ANNModel.fit(X_train, y_train,
              batch_size = batch_size,
              epochs = nb_epoch)

y_pred = ANNModel.predict_classes(X_test)

# print(y_pred)
# print(y_test)

predy = []
testy = []
for i in range(len(y_pred)):
    predy.append(y_pred[i][0])
    testy.append(y_test.loc[i])

Epoch 1/200
437/437 [==============================] - 0s 203us/step - loss: 0.8642 - accuracy: 0.4600
Epoch 2/200
437/437 [==============================] - 0s 30us/step - loss: 0.6872 - accuracy: 0.5767
Epoch 3/200
437/437 [==============================] - 0s 25us/step - loss: 0.6805 - accuracy: 0.5469
Epoch 4/200
437/437 [==============================] - 0s 32us/step - loss: 0.6778 - accuracy: 0.5423
Epoch 5/200
437/437 [==============================] - 0s 34us/step - loss: 0.6702 - accuracy: 0.6247
Epoch 6/200
437/437 [==============================] - 0s 30us/step - loss: 0.6540 - accuracy: 0.8398
Epoch 7/200
437/437 [==============================] - 0s 27us/step - loss: 0.6306 - accuracy: 0.7849
Epoch 8/200
437/437 [==============================] - 0s 27us/step - loss: 0.5965 - accuracy: 0.8055
Epoch 9/200
437/437 [==============================] - 0s 30us/step - loss: 0.5545 - accuracy: 0.8513
Epoch 10/200
437/437 [==============================] - 0s 32us/step - loss: 0.51

437/437 [==============================] - 0s 37us/step - loss: 0.2808 - accuracy: 0.8970
Epoch 160/200
437/437 [==============================] - 0s 27us/step - loss: 0.2795 - accuracy: 0.8993
Epoch 161/200
437/437 [==============================] - 0s 32us/step - loss: 0.2767 - accuracy: 0.9039
Epoch 162/200
437/437 [==============================] - 0s 30us/step - loss: 0.2774 - accuracy: 0.9085
Epoch 163/200
437/437 [==============================] - 0s 27us/step - loss: 0.2782 - accuracy: 0.9016
Epoch 164/200
437/437 [==============================] - 0s 30us/step - loss: 0.2937 - accuracy: 0.8924
Epoch 165/200
437/437 [==============================] - 0s 27us/step - loss: 0.2867 - accuracy: 0.9039
Epoch 166/200
437/437 [==============================] - 0s 25us/step - loss: 0.2798 - accuracy: 0.8993
Epoch 167/200
437/437 [==============================] - 0s 30us/step - loss: 0.2754 - accuracy: 0.8993
Epoch 168/200
437/437 [==============================] - 0s 27us/step - loss: 

In [167]:
accuracy = accuracy_score(testy, predy)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, predy)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, predy)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, predy)
print('F1 score: %f' % f1)

Accuracy: 0.870370
Precision: 0.897436
Recall: 0.867769
F1 score: 0.882353


### Q_2: Please list potential ethical bias inherent to solving the problem

- Bias can also be introduced into the data through how they are collected or selected for use. Oversampling certain neighborhoods because they are overpoliced can result in recording more crime, which results in less credit-worthiness.

- Older individuals have a higher likelihood of defaulting and reduces lending based on age, society and legal institutions may consider this to be illegal age discrimination.

Conclusion:  Setting different decision thresholds for different groups may achieve the best balance, particularly if we believe some of the underlying variables in the model may be biased. Different metrics and standards will likely be required, depending on the use case and circumstances.